In [ ]:
pip install contractions


In [ ]:
import boto3, sys,sagemaker                             
import pandas as pd   
import pandas as pd
import numpy as np
import nltk
import string
# import contractions
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
# plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket_name = 'howardvip-clinical-data'
def table(table_name, nrows=0):
    data_path = 's3://{}/{}'.format(bucket_name, 'NOTEEVENTS.csv.gz')
    if not nrows:
        return pd.read_csv(data_path)
    return pd.read_csv(data_path, nrows=nrows)


In [ ]:
df = table('NOTEEVENTS.csv.gz',nrows=5000)

In [ ]:
df

In [ ]:
dfText = df.loc[:,["TEXT"]]


In [ ]:
dfText["TEXT"][2]

In [ ]:
dfText['no_contract'] = dfText['TEXT'].apply(lambda x: [contractions.fix(word) for word in x.split()])
dfText.head()

In [ ]:
dfText["TEXT"] = [' '.join(map(str, l)) for l in  dfText['no_contract']]
dfText.head()

In [ ]:
nltk.download('punkt')
dfText['tokenized'] = dfText['TEXT'].apply(word_tokenize)


In [ ]:
dfText

In [ ]:
dfText['TEXT'][2]

In [ ]:
dfText['lower'] =dfText['tokenized'].apply(lambda x: [word.lower() for word in x])
dfText.head()

In [ ]:
punc = string.punctuation
dfText['no_punc'] = dfText['lower'].apply(lambda x: [word for word in x if word not in punc and word.isalpha()])
dfText.head()

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
dfText['stopwords_removed'] = dfText['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
dfText.head()

In [ ]:
nltk.download('averaged_perceptron_tagger')
dfText['pos_tags'] = dfText['stopwords_removed'].apply(nltk.tag.pos_tag)
dfText.head()

In [ ]:
nltk.download('wordnet')
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
dfText['wordnet_pos'] = dfText['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
dfText.head()

In [ ]:
wnl = WordNetLemmatizer()
dfText['lemmatized'] = dfText['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
dfText.head()

In [ ]:
dfText.to_csv('TEXT_clean.csv')

In [ ]:
dfText.to_csv('cleanedText.csv')

In [ ]:
dfText

In [ ]:
dfText['lemmatized'][1]

In [ ]:
dfText['TEXT'][1]

In [ ]:
import logging
from gensim.models.word2vec import Word2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
w2v_model = Word2Vec(dfText['lemmatized'], vector_size=100, min_count=2)


In [ ]:
w2v_model.wv.most_similar(['cancer'])

In [ ]:
w2v_model.wv.most_similar(['doctor'])

In [ ]:
w2v_model.wv.most_similar(['patient'])

In [ ]:
w2v_model.wv.most_similar(['disease'])

In [ ]:
w2v_model.save('modelwv')

In [ ]:
pip install gensim


In [ ]:
pip install gensim

In [ ]:
pip install python-Levenshtein

In [ ]:
from gensim.models.word2vec import Word2Vec
model1 = Word2Vec.load('modelwv')
model2 = Word2Vec.load('w2vec2')

In [ ]:
model1.wv.most_similar(['disease'])

In [ ]:
model1.wv.most_similar(['heart'])

In [ ]:
model2.wv.most_similar(['heart'])

In [ ]:
model2.wv.most_similar(['allergy'])

In [ ]:
model2.wv.most_similar(['disease'])

In [ ]:
model1.wv.most_similar(['leukemia'])

In [ ]:
model2.wv.most_similar(['leukemia'])

In [ ]:
model1.wv.most_similar(['physician'])

In [ ]:
model2.wv.most_similar(['physician'])

In [ ]:
word_vectors1 = model1.wv
word_vectors2 = model2.wv

In [ ]:
word_vectors1.save("word2vec.wordvectors1")
word_vectors2.save("word2vec.wordvectors2")

In [ ]:
from gensim.models import KeyedVectors
wv1 = KeyedVectors.load("word2vec.wordvectors1", mmap='r')
wv2 = KeyedVectors.load("word2vec.wordvectors2", mmap='r')

In [ ]:
wv1["patient"]

In [ ]:
wv2["patient"]

In [ ]:
wv1.most_similar(positive=['patient', 'list(w2.key_to_index.keys())[-25:])'], negative=['sick'])

In [ ]:
wv1.most_similar(positive=['pressure', "depression"], negative=["drug", "medicine"])

In [ ]:
wv2.most_similar(positive=['pressure', "depression"], negative=["drug", "medicine"])

In [ ]:
list(wv1.key_to_index.keys())[:-20]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import cluster
from sklearn import metrics
from gensim.models.word2vec import Word2Vec

In [ ]:
df = pd.read_csv('sentenceRowClean.csv', index_col=0)
df

In [ ]:
y = df['lemmatized'].tolist()
sentences = []
m = ""
for row in y:
    seen = False
    for i in row:
        if i == "[":
             seen = True
        elif i == "]":
            sentences.append(m.split(","))
            m = ""
            seen = False
        elif seen and i !="'" and i != "]" and i != "[" and i !=" ":
            m += i

In [ ]:

sentences[3]

In [ ]:
y[0]

In [ ]:
import logging
from gensim.models.word2vec import Word2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
w2v_model2 = Word2Vec(sentences, vector_size=100, min_count=2)

In [ ]:
w2v_model2.wv.most_similar(['physician'])

In [ ]:
w2v_model2.wv.most_similar(['disease'])

In [ ]:
import numpy as np 
def sent_vectorizer(sent, model):
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model.wv[w]
            else:
                sent_vec = np.add(sent_vec, model.wv[w])
            numw+=1
        except:
            pass
     
    return np.asarray(sent_vec) / numw


In [ ]:
X = []
for sentence in sentences:
    m = (sent_vectorizer(sentence, w2v_model2))
    if len(m) > 0:
        X.append(m)

In [ ]:
print (model1.wv[model1.wv.key_to_index])
# print (model.similarity('post', 'book'))
# print (model.most_similar(positive=['machine'], negative=[], topn=2))

In [ ]:
len(X)

In [ ]:
kmeans = cluster.KMeans(n_clusters=2)
kmeans.fit(X)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')

In [ ]:
import nltk
from nltk.cluster import KMeansClusterer
NUM_CLUSTERS=2
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(X[:5000], assign_clusters=True)

In [ ]:
from sklearn.manifold import TSNE
 
model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
 
Y=model.fit_transform(X[:5000])
 
plt.scatter(Y[:, 0], Y[:, 1], c=assigned_clusters, s=290,alpha=.5)
for j in range(5000):    
#    plt.annotate(assigned_clusters[j],xy=(Y[j][0], Y[j][1]),xytext=(0,0),textcoords='offset points')
   print ("%s %s" % (assigned_clusters[j],  sentences[j]))
 
plt.show()

In [ ]:
def tsne_plot(model):
    labels = []
    tokens = []

    for word in model.wv.key_to_index:
        tokens.append(model.wv[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()
tsne_plot(w2v_model2)

In [ ]:
print(assigned_clusters)

for j in range(len(sentences)):    
   plt.annotate(assigned_clusters[j],xy=(Y[j][0], Y[j][1]),xytext=(0,0),textcoords='offset points')
   print (assigned_clusters[j],  sentences[j])

In [ ]:
print(sentences)

In [ ]:
print(sentences[2])